In [ ]:
!sudo apt-get update
!sudo apt-get install -y x13as

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,610 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,452 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,4

In [ ]:
!apt-get install octave &> /dev/null

In [ ]:
!apt-get install dynare &> /dev/null

In [ ]:
%%writefile rbc_no_gov.mod

var c h k w r y i A;
varexo e;
parameters phi psi sigma alpha beta delta rho;

phi = 1;
psi = 2.29;
sigma = 2;
alpha = 0.44;
beta = 0.97;
delta = 0.05;
rho = 0.9;

model;

# Abar = 1;
# rbar = 1 / beta - 1 + delta;
# koh = ( rbar / alpha )^( 1 / ( alpha - 1 ) );
# coh = koh^alpha - delta * koh;
# hbar = ( ( 1 - alpha ) / psi * koh^alpha * coh^(-sigma) )^( 1 / ( phi + sigma - 1 ) );
# kbar = koh * hbar;
# ybar = Abar * kbar^alpha * hbar^(1-alpha);

psi * exp(h)^phi * exp(c)^sigma = exp(w);
exp(c)^(-sigma) = beta * ( exp(c(+1)) )^(-sigma) * ( 1 + exp(r(+1)) - delta );
exp(k) = ( 1 - delta ) * exp(k(-1)) + exp(i);
exp(y) = exp(A) * exp(k(-1))^alpha * exp(h)^(1-alpha);
exp(r) = alpha * exp(y) / exp(k);
exp(w) = (1 - alpha) * exp(y) / exp(h);
exp(y) = exp(c) + exp(i);
A = (1 - rho) * Abar + rho * A(-1) + e;

end;

steady;

check;

shocks;
var e;
stderr 1;
end;

stoch_simul(ar=1, order=1, irf=20, graph_format = pdf);

Overwriting rbc_no_gov.mod


In [ ]:
%%writefile solve.m
dynare rbc_no_gov.mod

Overwriting solve.m


In [ ]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 8 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 1).
Processing outputs ...
done
Preprocessing completed.


STEADY-STATE RESULTS:

c 		 1.50219
h 		 -1.29666
k 		 3.51264
w 		 2.53628
r 		 -2.51419
y 		 1.81943
i 		 0.51691
A 		 1

EIGENVALUES:
         Modulus             Real        Imaginary

             0.9              0.9                0
          0.9428           0.9428                0
           1.105            1.105                0
       1.241e+17        1.241e+17                0


There are 2 eigenvalue(s) larger than 1 in modulus 
for 2 forward-looking variable(s)

The rank condition is verified.


MODEL SUMMARY

  Number of variables:         8
  Number of stochastic shocks: 1
  Number of 

In [ ]:
%%writefile rbc_gov.mod

var c h k w r y i A G;
varexo eA eG;
parameters phi psi sigma alpha beta delta rho gs rho_G;

phi = 1;
psi = 2.29;
sigma = 2;
alpha = 0.44;
beta = 0.97;
delta = 0.05;
rho = 0.9;
gs = 0.2;
rho_G = 0.9;

model;

# Abar = 1;
# rbar = 1 / beta - 1 + delta;
# koh = ( rbar / alpha )^( 1 / ( alpha - 1 ) );
# coh = ( 1 - gs ) * koh^alpha - delta * koh;
# hbar = ( ( 1 - alpha ) / psi * koh^alpha * coh^(-sigma) )^( 1 / ( phi + sigma - 1 ) );
# kbar = koh * hbar;
# ybar = Abar * kbar^alpha * hbar^(1-alpha);
# Gbar = gs * ybar;

psi * exp(h)^phi * exp(c)^sigma = exp(w);
exp(c)^(-sigma) = beta * ( exp(c(+1)) )^(-sigma) * ( 1 + exp(r(+1)) - delta );
exp(k) = ( 1 - delta ) * exp(k(-1)) + exp(i);
exp(y) = exp(A) * exp(k(-1))^alpha * exp(h)^(1-alpha);
exp(r) = alpha * exp(y) / exp(k);
exp(w) = (1 - alpha) * exp(y) / exp(h);
exp(y) = exp(c) + exp(i) + exp(G);

G = (1 - rho_G) * Gbar + rho_G * G(-1) + eG;
A = (1 - rho) * Abar + rho * A(-1) + eA;

end;

steady;

check;

shocks;
var eA;
stderr 1;
var eG;
stderr 1.99;
end;

stoch_simul(ar=1, order=1, irf=20, graph_format=pdf);

Overwriting rbc_gov.mod


In [ ]:
%%writefile solve.m
dynare rbc_gov.mod

Overwriting solve.m


In [ ]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 9 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 1).
Processing outputs ...
done
Preprocessing completed.


STEADY-STATE RESULTS:

c 		 1.40103
h 		 -1.09434
k 		 3.71498
w 		 2.53628
r 		 -2.5142
y 		 2.02176
i 		 0.719249
A 		 1
G 		 0.364197

EIGENVALUES:
         Modulus             Real        Imaginary

             0.9              0.9                0
             0.9              0.9                0
          0.9455           0.9455                0
           1.101            1.101                0
       2.212e+14       -2.212e+14                0


There are 2 eigenvalue(s) larger than 1 in modulus 
for 2 forward-looking variable(s)

The rank condition is verified.


MODEL SUMMARY

  Number of v

In [ ]:
%%writefile rbc_gov_cm.mod

var c h k w r y i A G;
varexo eA eG;
parameters phi psi sigma alpha beta delta rho gs rho_G mu;

phi = 1;
psi = 2.29;
sigma = 2;
alpha = 0.44;
beta = 0.97;
delta = 0.05;
rho = 0.9;
gs = 0.2;
rho_G = 0.9;
mu = 1.2; // Markup para refletir concorrência monopolística

model;

# Abar = 1;
# rbar = 1 / beta - 1 + delta;
# koh = ( rbar / alpha )^( 1 / ( alpha - 1 ) );
# coh = ( 1 - gs ) * koh^alpha - delta * koh;
# hbar = ( ( 1 - alpha ) / psi * koh^alpha * coh^(-sigma) )^( 1 / ( phi + sigma - 1 ) );
# kbar = koh * hbar;
# ybar = Abar * kbar^alpha * hbar^(1-alpha);
# Gbar = gs * ybar;

psi * exp(h)^phi * exp(c)^sigma = exp(w);
exp(c)^(-sigma) = beta * ( exp(c(+1)) )^(-sigma) * ( 1 + exp(r(+1)) - delta );
exp(k) = ( 1 - delta ) * exp(k(-1)) + exp(i);
exp(y) = exp(A) * exp(k(-1))^alpha * exp(h)^(1-alpha);
exp(r) = alpha * exp(y) / (mu * exp(k));
exp(w) = (1 - alpha) * exp(y) / (mu * exp(h));
exp(y) = exp(c) + exp(i) + exp(G);

G = (1 - rho_G) * Gbar + rho_G * G(-1) + eG;
A = (1 - rho) * Abar + rho * A(-1) + eA;

end;

steady;

check;

shocks;
var eA;
stderr 1;
var eG;
stderr 1.99;
end;

stoch_simul(ar=1, order=1, irf=20, graph_format=pdf);

Overwriting rbc_gov_cm.mod


In [ ]:
%%writefile solve.m
dynare rbc_gov_cm.mod

Overwriting solve.m


In [ ]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 9 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 1).
Processing outputs ...
done
Preprocessing completed.


STEADY-STATE RESULTS:

c 		 1.25105
h 		 -1.11994
k 		 3.36379
w 		 2.21071
r 		 -2.5142
y 		 1.8529
i 		 0.368062
A 		 1
G 		 0.364197

EIGENVALUES:
         Modulus             Real        Imaginary

             0.9              0.9                0
             0.9              0.9                0
          0.9452           0.9452                0
           1.126            1.126                0
       1.898e+15       -1.898e+15                0


There are 2 eigenvalue(s) larger than 1 in modulus 
for 2 forward-looking variable(s)

The rank condition is verified.


MODEL SUMMARY

  Number of va